In [ ]:
# default_exp arch

In [ ]:
# export 
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

## 1. Init function and single Layers

In [ ]:
# export

def weights_init(m):
    """
    Define the weight parameters depending on the type:
    Conv or Batchnorm
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# export

class BlurPool3d(nn.Module):
    """
    Modification of: https://arxiv.org/abs/1904.11486 for 3d purpose.
    Copyright (c) 2019, Adobe Inc. All rights reserved.
    """

    def __init__(self, filt_size=3, stride=2, channels=None, pad_off=0):

        super(BlurPool3d, self).__init__()
        self.filt_size = filt_size
        self.pad_off = pad_off
        self.pad_sizes = [int(1.*(filt_size-1)/2), int(np.ceil(1.*(filt_size-1)/2)),
                          int(1.*(filt_size-1)/2), int(np.ceil(1.*(filt_size-1)/2)),
                          int(1.*(filt_size-1)/2), int(np.ceil(1.*(filt_size-1)/2))]
        self.pad_sizes = [pad_size+pad_off for pad_size in self.pad_sizes]
        self.stride = stride
        self.channels = channels

        if(self.filt_size == 1):
            a = np.array([1., ])
        elif(self.filt_size == 2):
            a = np.array([1., 1.])
        elif(self.filt_size == 3):
            a = np.array([1., 2., 1.])
        elif(self.filt_size == 4):
            a = np.array([1., 3., 3., 1.])
        elif(self.filt_size == 5):
            a = np.array([1., 4., 6., 4., 1.])
        elif(self.filt_size == 6):
            a = np.array([1., 5., 10., 10., 5., 1.])
        elif(self.filt_size == 7):
            a = np.array([1., 6., 15., 20., 15., 6., 1.])

        filt = torch.Tensor(
            a[:, None, None] * a[None, None, :] * a[None, :,  None])
        filt = filt/torch.sum(filt)

        self.register_buffer(
            'filt', filt[None, None, :, :, :].repeat((self.channels, 1,  1, 1, 1)))
        # Only pad possible for 3d use
        self.pad = nn.ReplicationPad3d(self.pad_sizes)

    def forward(self, x):
        return F.conv3d(self.pad(x), self.filt, stride=self.stride, groups=x.shape[1])

In [ ]:
# export

class Quantize(nn.Module):
    """
    Quantization 'Layer'
    inspired by: https://github.com/deepmind/sonnet
    modified from: https://github.com/rosinality/vq-vae-2-pytorch
    """

    def __init__(self, dim, n_embed, decay=0.99, eps=1e-5):
        super().__init__()
        """
        Setup the embedding Matrix (dim x n_embed)
        """
        # This is equal to the feature number at the corresponding layer
        self.dim = dim
        # This is the discretization level for each pixel
        self.n_embed = n_embed
        # Learning parameters
        self.decay = decay
        self.eps = eps
        # Init matrix of available embeddings
        embed = torch.randn(dim, n_embed)
        # Register to avoid gradients
        self.register_buffer('embed', embed)
        self.register_buffer('cluster_size', torch.zeros(n_embed))
        self.register_buffer('embed_avg', embed.clone())

    def forward(self, x, update=True):
        """
        Apply the Quantization on the input:
        -> Update Embeddings
        -> Remain the Gradient flow
        Return:
            Quantized input
            Difference between true and
        """
        # Reshape input
        flatten = x.reshape(-1, self.dim)
        # Calculate L2-distance between each pixel / voxel and embedding
        dist = (
            flatten.pow(2).sum(1, keepdim=True)
            - 2 * flatten @ self.embed
            + self.embed.pow(2).sum(0, keepdim=True)
        )
        # Select the closest pairs
        _, embed_ind = (-dist).max(1)
        # Construct matrix of matching pairs
        embed_onehot = F.one_hot(embed_ind, self.n_embed).type(flatten.dtype)
        # Select the correct dimensions here!
        embed_ind = embed_ind.view(*x.shape[:-1])
        # Apply quantization
        quantize = self.embed_code(embed_ind)

        # Only for training -> Update Embeddings with moving average
        if update:
            # N_i = N_(i-1) * gamma + (1-gamma) * n_i
            self.cluster_size.data.mul_(self.decay).add_(
                1 - self.decay, embed_onehot.sum(0)
            )
            # Sum(E(x))
            embed_sum = flatten.transpose(0, 1) @ embed_onehot
            # m_i = m_(i-1) *gamma + (1-gamma) * Sum(E(x))
            self.embed_avg.data.mul_(self.decay).add_(
                1 - self.decay, embed_sum)
            # N_i
            n = self.cluster_size.sum()
            # norm N_i
            cluster_size = (
                (self.cluster_size + self.eps) /
                (n + self.n_embed * self.eps) * n
            )
            embed_normalized = self.embed_avg / cluster_size.unsqueeze(0)
            # e_i = m_i / N_i
            self.embed.data.copy_(embed_normalized)

        # Loss between Original to Quantization
        diff = (quantize.detach() - x).pow(2).mean()
        # Get Output, while enabling to copy gradients
        quantize = x + (quantize - x).detach()
        # Return results
        return quantize, diff, embed_ind

    def embed_code(self, embed_id):
        """
        Perform the quantization by selecting all embedings from the ids
        """
        return F.embedding(embed_id, self.embed.transpose(0, 1))

In [ ]:
quant = Quantize(64, 100)
input = torch.randn(16, 16, 8, 64)
quantize, diff, embed_ind = quant(input)
print(quantize.shape, diff, embed_ind.shape)

torch.Size([16, 16, 8, 64]) tensor(1.3022) torch.Size([16, 16, 8])


## 2. Block of Layers

In [ ]:
# export

class ResNetBlock(nn.Module):
    """
    An individually designalble ResNet Block for 3 Dimensional Convoluions
    based on: https://arxiv.org/abs/1512.03385
    """

    def __init__(self, n_chan, convsize=3,
                 activation=nn.ReLU(inplace=True),
                 init_w=weights_init, dim=3):
        """setup the general architecture"""
        super(ResNetBlock, self).__init__()

        if dim == 3:
            Conv = nn.Conv3d
            BatchNorm = nn.BatchNorm3d
        else:
            Conv = nn.Conv2d
            BatchNorm = nn.BatchNorm2d

        self.activation = activation

        self.main_part = nn.Sequential(
            # Layer 1
            Conv(n_chan, n_chan, convsize,
                 stride=1, padding=1, bias=False),
            BatchNorm(n_chan),
            self.activation,
            # Layer 2
            Conv(n_chan, n_chan, convsize,
                 stride=1, padding=1, bias=False),
            BatchNorm(n_chan),
        )
        self.main_part.apply(init_w)

    def forward(self, x):
        """Calculate the forward pass"""
        residual = x
        out = self.main_part(x)
        out += residual
        out = self.activation(out)

        return out

In [ ]:
# export

class ConvBn(nn.Module):
    """
    An individually designalble Block for 3 Dimensional Convoluions with Batchnorm and Dropout
    """

    def __init__(self, in_chan, out_chan, convsize=3,
                 stride=2, activation=nn.LeakyReLU(0.2, inplace=True),
                 init_w=weights_init, padding=1, dim=3, p_drop=0):
        """setup the general architecture"""
        super(ConvBn, self).__init__()

        if dim == 3:
            Conv = nn.Conv3d
            BatchNorm = nn.BatchNorm3d
            Dropout = nn.Dropout3d
        else:
            Conv = nn.Conv2d
            BatchNorm = nn.BatchNorm2d
            Dropout = nn.Dropout2d
            # Check convsize and stride
            if type(convsize) == tuple:
                if len(convsize) > 2:
                    convsize = convsize[1:]
            if type(stride) == tuple:
                if len(stride) > 2:
                    stride = stride[1:]

        self.main_part = nn.Sequential(
            Conv(in_chan, out_chan, convsize,
                 stride=stride, padding=padding, bias=False),
            BatchNorm(out_chan),
            activation,
            Dropout(p=p_drop),
        )
        self.main_part.apply(init_w)

    def forward(self, x):
        """Calculate the forward pass"""
        return self.main_part(x)

In [ ]:
# Test it!
m = ConvBn(1, 16, convsize=4, stride=(2, 2, 2), padding=(1, 1, 1))
inp = torch.randn(20, 1, 16, 64, 64)
output = m(inp)
print(output.shape)

torch.Size([20, 16, 8, 32, 32])


In [ ]:
# export

class ConvTpBn(nn.Module):
    """
    An individually designalble Block for 3 Dimensional Transposed Convoluions with Batchnorm
    """

    def __init__(self, in_chan, out_chan, convsize=3,
                 stride=2, activation=nn.ReLU(inplace=True),
                 init_w=weights_init, padding=1, dim=3):
        """setup the general architecture"""
        super(ConvTpBn, self).__init__()
        if dim == 3:
            ConvTranspose = nn.ConvTranspose3d
            BatchNorm = nn.BatchNorm3d
        else:
            ConvTranspose = nn.ConvTranspose2d
            BatchNorm = nn.BatchNorm2d
            # Check convsize and stride
            if type(convsize) == tuple:
                if len(convsize) > 2:
                    convsize = convsize[1:]
            if type(stride) == tuple:
                if len(stride) > 2:
                    stride = stride[1:]

        self.main_part = nn.Sequential(
            ConvTranspose(in_chan, out_chan, convsize,
                          stride=stride, padding=padding, bias=False),
            BatchNorm(out_chan),
            activation,
        )
        self.main_part.apply(init_w)

    def forward(self, x):
        """Calculate the forward pass"""
        return self.main_part(x)

In [ ]:
m = ConvTpBn(1, 16, convsize=3, stride=(2, 2, 2), padding=(1, 1, 1))
output = m(inp)
print(output.shape)

torch.Size([20, 16, 31, 127, 127])


In [ ]:
# export

class LinearSigmoid(nn.Module):
    """
    Helper class to provide a simple ending with linear layer and Sigmoid
    """

    def __init__(self, hidden_dim, y_dim, bias=False):
        """setup network"""
        super(LinearSigmoid, self).__init__()
        # store inputs
        self.hidden_dim = hidden_dim
        self.y_dim = y_dim
        self.bias = bias
        # make network
        self.main_part = nn.Sequential(
            nn.Linear(self.hidden_dim, self.y_dim, bias=self.bias),
            nn.Sigmoid()
        )

    def forward(self, x):
        """reformat then apply fc part"""
        x = x.view((-1, self.hidden_dim))
        x = self.main_part(x)
        return x

## 3 The generic convolutional network block

In [ ]:
# export
class DownUpConv(nn.Module):
    """
    A helper Type, which contains the generic conv network
    for 3d up- or downsclaing depending on "move"
    """

    def __init__(self, args, n_fea_in, n_fea_next, pic_size, depth, move="down", p_drop=0):
        """Setup the conv-network"""
        super(DownUpConv, self).__init__()
        # Convolutions in 2d / 3d
        self.dim = args.dim
        # Input dim
        self.pic_size = pic_size
        self.depth = depth
        self.n_fea_in = n_fea_in
        # Number of feature channels
        self.n_fea_next = n_fea_next
        # Output dim
        self.min_size = args.min_size  # when to stop reducing
        # Scaling
        self.scale2d = args.scale2d
        self.scale3d = args.scale3d
        self.n_res2d = args.n_res2d
        self.n_res3d = args.n_res3d
        # Dropout
        self.p_drop = p_drop
        # Add the relevant quantization layers if required
        self.vq_layers = args.vq_layers if args.model_type == "vqvae" else []
        # Direction
        self.move = move  # Define whether to scale up or down
        self.main, output_tuple = self.generic_conv_init()
        self.max_fea, self.max_fea_next, self.pic_out, self.final_depth = output_tuple

    def add_layers(self, conv_layers, fea_in, fea_out, n_res, pic_size, convsize=4, stride=2):
        """
        Add layers according to number of residual blocks and down/upscale mode
        """
        # Downsample
        # ---------------------------------------------------
        if self.move == "down":
            # 1. Downsample
            conv_layers.extend([
                ConvBn(fea_in, fea_out, dim=self.dim,
                       convsize=convsize, stride=stride, p_drop=self.p_drop),
            ])
            # 2. Add the residual blocks:
            for _ in range(n_res):
                conv_layers.extend([
                    ResNetBlock(fea_out, convsize=3, dim=self.dim),
                ])

        # Upsample
        # ---------------------------------------------------
        else:
            # Attention! This layer can be a quantization layer!
            special_pic = pic_size / 2
            if special_pic > self.min_size and special_pic in self.vq_layers:
                fea_out *= 2  # so it has a doubled feature size

            # 1. Add the residual blocks:
            for _ in range(n_res):
                conv_layers[:0] = ([
                    ResNetBlock(fea_in, convsize=3, dim=self.dim),
                ])

            # 2. Upsample
            conv_layers[:0] = ([
                ConvTpBn(fea_out, fea_in, dim=self.dim,
                         convsize=convsize, stride=stride),
            ])

    def generic_conv_init(self):
        """
        Initialise the convolution layers generically:
        Down:
        Idea: -> Half picsize until all 3 dims are equal
              -> Then reduce all dims until min_size.
        Up:
        Idea: -> Double all dims until z-limit is reached
              -> Then double picsize until output-dim is reached.
        """

        # Init the conv_layer list
        conv_layers = []
        # Current z-dim of the picture
        cur_pic_dim = self.pic_size
        # Current anz of features at input size
        cur_fea_in = self.n_fea_in
        # Current anz of features at output size
        cur_fea_out = self.n_fea_next
        # Current depth of the picture
        cur_depth = self.depth
        # Summarize current values
        output_tuple = (cur_fea_in, cur_fea_out, cur_pic_dim, cur_depth)

        # Until the limiting z_dim occurs or picsize too small
        # ---------------------------------------------------------------
        while cur_pic_dim > self.depth and cur_pic_dim > self.min_size:
            # Add layers
            self.add_layers(
                conv_layers, cur_fea_in, cur_fea_out, self.n_res2d,
                cur_pic_dim, convsize=(3, 4, 4), stride=(1, 2, 2))
            # Update input size
            cur_fea_in = cur_fea_out
            # Features are doupled
            cur_fea_out *= self.scale2d
            cur_pic_dim /= 2  # dimension is halved

            # Store current values
            output_tuple = (cur_fea_in, cur_fea_out, cur_pic_dim, cur_depth)

            # CASE: Layer is a quantization layer!!
            if cur_pic_dim in self.vq_layers:
                # Return current Network and relevant parameters
                return nn.Sequential(*conv_layers), output_tuple

        # adjust for new scale factor
        # cur_fea_out = max([cur_fea_in * self.scale3d, self.n_fea_next])

        # Limit reached, now continue until min-picsize reached
        # ---------------------------------------------------------------
        while cur_pic_dim > self.min_size:
            # Add layers
            self.add_layers(
                conv_layers, cur_fea_in, cur_fea_out, self.n_res3d,
                cur_pic_dim, convsize=4, stride=2)
            # Update input size
            cur_fea_in = cur_fea_out
            # Features are doupled
            cur_fea_out *= self.scale3d
            cur_pic_dim /= 2  # Dimension is halved
            cur_depth /= 2  # Depth is also halfed

            # Store current values
            output_tuple = (cur_fea_in, cur_fea_out, cur_pic_dim, cur_depth)

            # CASE: Layer is a quantization layer!!
            if cur_pic_dim in self.vq_layers:
                # Return current Network and relevant parameters
                return nn.Sequential(*conv_layers), output_tuple

        # Finally return the network at minimum size
        return nn.Sequential(*conv_layers), output_tuple

    def forward(self, x):
        """Calculate the forward pass"""
        return self.main(x)

## 4. Whole Networks

In [ ]:
# export

class Encoder(nn.Module):
    """Encoder with 3dimensional conv setup"""

    def __init__(self, args, init_w=weights_init, vae_mode=True):
        """
        Setup the Architecture:
        Args:
            ngpu = Number of GPUs available
            init_w = Function for initialisation of weights
            n_chan = Number of input channels: batch x n_chan x depth x size x size
            n_d_fea = Number of feature channels within network
            n_z = Latent space dimension
        """
        super(Encoder, self).__init__()
        # vae / ae definitions
        self.n_z = args.n_z * 2 if vae_mode else args.n_z
        self.forward = self.forward_vae if vae_mode else self.forward_ae

        # Convolutional network
        self.conv_part = DownUpConv(args, n_fea_next=args.n_fea_down, move="down",
                                    pic_size=args.pic_size, depth=args.crop_size,
                                    n_fea_in=len(args.perspectives))
        self.max_fea = self.conv_part.max_fea
        self.hidden_dim = self.max_fea * args.min_size**(args.dim)

        # Finish with fully connected layers
        self.fc_part = nn.Sequential(
            # State size batch x (cur_fea*4*4*4)
            nn.Linear(self.hidden_dim, self.n_z, bias=False),
            nn.ReLU(inplace=True),
            # Output size batch x n_z
        )
        # Initialise (conv part is already)
        self.fc_part.apply(init_w)

    def forward_vae(self, x):
        """calculate output, return mu and sigma"""
        # Apply convolutions
        x = self.conv_part(x)
        # Resize
        x = x.view((-1, self.hidden_dim))
        # Apply fully connected part
        x = self.fc_part(x)
        # Separate mu and sigma
        mu, logvar = x.chunk(2, dim=1)
        return mu, logvar

    def forward_ae(self, x):
        """calculate output, return mu and sigma"""
        # Apply convolutions
        x = self.conv_part(x)
        # Resize
        x = x.view((-1, self.hidden_dim))
        # Apply fully connected part
        x = self.fc_part(x)
        return x

In [ ]:
# export

class Decoder(nn.Module):
    """Decoder class (also a Generator)"""

    def __init__(self, args, init_w=weights_init):
        """
        Setup the Architecture:
        Args:
            init_w = Function for initialisation of weights
            n_chan = Number of input channels: batch x n_chan x depth x size x size
            n_d_fea = Number of feature channels within network
            n_z = Latent space dimension
        """
        super(Decoder, self).__init__()

        # Convolutional network
        self.conv_part = DownUpConv(args, n_fea_next=args.n_fea_up, move="up",
                                    pic_size=args.pic_size, depth=args.crop_size,
                                    n_fea_in=len(args.perspectives))
        self.max_fea = self.conv_part.max_fea
        self.hidden_dim = self.max_fea * args.min_size**(args.dim)
        self.view_arr = [-1, self.max_fea]
        self.view_arr.extend([args.min_size for _ in range(args.dim)])

        self.fc_part = nn.Sequential(
            # Input is batch x n_z
            nn.Linear(args.n_z, self.hidden_dim, bias=False),
            # nn.Tanh(),
        )
        # Initialise (conv part is already)
        self.fc_part.apply(init_w)

    def forward(self, x):
        """calculate output"""
        # Apply fully connected part
        x = self.fc_part(x)
        # Resize
        x = x.view(self.view_arr)
        # Apply convolutions
        x = self.conv_part(x)
        return x

In [ ]:
# export

class Discriminator(nn.Module):
    """
    Discriminator class, only for true/fake differences
    Classifier for Determinig between several classes
    """

    def __init__(self, args, diag_dim=1, init_w=weights_init, wgan=False):
        """Setup the Architecture:"""
        super(Discriminator, self).__init__()

        # Convolutional network
        self.conv_part = DownUpConv(args, n_fea_next=args.n_fea_down, move="down",
                                    pic_size=args.pic_size, depth=args.crop_size,
                                    n_fea_in=len(args.perspectives), p_drop=args.p_drop)
        self.max_fea = self.conv_part.max_fea
        self.hidden_dim = self.max_fea * args.min_size**(args.dim)

        # Finish with fully connected layers
        self.fc_part = nn.Sequential(
            # State size batch x (cur_fea*4*4*4)
            nn.Linear(self.hidden_dim, diag_dim, bias=False),
        )
        # Initialise (conv part is already)
        self.fc_part.apply(init_w)
        self.forward = self.forward_wgan if wgan else self.forward_dcgan

    def forward_wgan(self, x):
        """calculate output, return prob real / fake"""

        # Apply convolutions
        x = self.conv_part(x)
        # Resize
        x = x.view((-1, self.hidden_dim))
        # Apply fully connected part
        x = self.fc_part(x)
        return x

    def forward_dcgan(self, x):
        """calculate output, return prob real / fake"""

        # Apply Network
        x = self.forward_wgan(x)
        torch.sigmoid_(x)
        return x

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_demo.ipynb.
Converted 01_architecture.ipynb.
Converted 99_index.ipynb.
